# The Machine Learning Group Challenge!

##  Today's Mission
- Your objective is to devise the best possible model to predict successful/default loans using a preprocessed version of the Lending Club loan dataset.

- Class into divided into 4 groups.

- The training data is 13689 loans labeled either as 1 (successful) or 0 (default). Comes with 30 categorical and numerical features. The testing data is also 13689 loans.

- A data dictionary file is included as well. It is a table explaining each what each feature means.

- Groups will judged on how much money their model makes. You will evaluate your model on the testing dataset. You cannot used testing data to train a model.

- Your profit will be determined by the amount of money you make from correctly predicting good loans (loan amount * interest rate/100.) subtracted by the money you lose from incorrectly predicting bad loans as good (-loan amount). I have given a function to calculate that.

- Finding the correct probability threshold will be super key in determing how much money you make.

- **VERY IMPORTANT:** You must transform your testing the data the same way you did your training data. So make sure you have the same columns in the same order.
        
- Stewart and I will be on be hand for guidance. However I want you to primarily use your teammates for help. 

- Use all the tools at your disposal, try all the models we've learned in class. Refer to past class notebooks for help. Be sure to use modeling evaluating techniques such as ROC curves, confusion matrix, recall/precision, etc.

- To optimize model, find the right combination of features and the right model with the right parameters. Get creative!

- Remember to use your time wisely, it will go by fast. Communicate amongst yourselves often.
   

### Online resources on Lending Club loan data
Kaggle Page: https://www.kaggle.com/wendykan/lending-club-loan-data. Make sure to check out the kernels section.

Y Hat tutorial (It's in R, but its still useful): http://blog.yhat.com/posts/machine-learning-for-predicting-bad-loans.html

Blog tutorial on the data from Kevin Davenport: http://kldavenport.com/lending-club-data-analysis-revisted-with-python/



### Class Time
No class breaks. But individual breaks are allowed of course.

- 6:30 - 7:00
    - Feature engineering/selection: make dummy variables, dropping features, scaling, and other methods of transforming data. 
    - Exploratory data analysis aka get to know your features time.
    
    
- 7:00 - 8:45
    - Modeling time!!
    
    
- 8:45 - 9:25
    - Model testing.
    
    
- 9:25 - 9:30
    - Winner decision. Exit tickets.

In [1]:
#Imports and set pandas options
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
pd.set_option("max.columns", 100)
pd.set_option("max.colwidth", 100)

In [2]:
# Load in training data.
# Loan_status column is the target variable. Remember to drop it from df.
train = pd.read_csv("../../data/lending_club/challenge_training_data.csv")
train.head(2)

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,acc_now_delinq,delinq_amnt,tax_liens
0,742238,3150.0,3150.0,36 months,10.99%,103.12,B,A-1 Casino Rd Mini Storage,9 years,RENT,15300.0,Verified,1,Borrower added on 05/02/11 > I plan to use these funds to pay off a higher rate credit card.<b...,debt_consolidation,lowerinterest,19.76,0.0,695.0,699.0,0.0,5.0,0.0,9316.0,76.4%,9.0,Feb-2014,0.0,0.0,0.0
1,567423,10000.0,10000.0,36 months,7.51%,311.11,A,ORION MECHANICAL INC.,4 years,OWN,125000.0,Not Verified,1,Borrower added on 08/19/10 > CONSOLIDATE OUT STANDING CREDIT CARD LOANS AND PURCHASES FOR NEW ...,debt_consolidation,CONSOLIDATE,18.43,0.0,800.0,804.0,1.0,16.0,0.0,25364.0,2.9%,40.0,Feb-2017,0.0,0.0,0.0


In [4]:
#Load in data dictionary
data_dict = pd.read_csv("../../data/lending_club/the_data_dictionary.csv")
data_dict

,dtypes,name,description
0,object,id,A unique LC assigned ID for the loan listing.
1,float64,loan_amnt,"The listed amount of the loan applied for by the borrower. If at some point in time, the credit ..."
2,float64,funded_amnt,The total amount committed to that loan at that point in time.
3,object,term,The number of payments on the loan. Values are in months and can be either 36 or 60.
4,object,int_rate,Interest Rate on the loan
5,float64,installment,The monthly payment owed by the borrower if the loan originates.
6,object,grade,LC assigned loan grade
7,object,emp_title,The job title supplied by the Borrower when applying for the loan.*
8,object,emp_length,Employment length in years. Possible values are between 0 and 10 where 0 means less than one yea...
9,object,home_ownership,The home ownership status provided by the borrower during registration or obtained from the cred...


In [ ]:
#Load in test data when you're ready 
# test = pd.read_csv("../../data/lending_club/challenge_testing_data.csv")

Here is the threshold function. Input your probabilities for class 1 and set a probability threshold of your choice. The default threshold is 0.5. The output will be 1's and 0s, 1 values for all the values that are greater or equal to your predetermined threshold value.

In [5]:

def threshold(probs, thres = 0.5):
    output = np.where(probs >= thres, 1, 0)
    return output

This is the profit function. It takes in a dataframe with the loan_amnt, int_rate, target variable, and class predictions values.

- It first checks to see if a row has 0 in the outcome column and 1 in the predicted (false positive) and returns the negative value of the loan_amnt. This is how much money you lost for loans that mean that condition.

- Then it checks for true positives, meaning conditions where both the target and predicted values equal 1, then return the loan_amnt times the int_rate divided by 100. This is how much money you made from loans that meet this condition.

- Everything else gets a zero.

In [6]:
def profit_function(data):
    if data.target == 0 and data.predicted == 1:
        return -1 *data.loan_amnt
    elif data.target == 1 and data.predicted == 1:
        return data.loan_amnt * (data.int_rate/100.)
    else:
        return 0

Here's an example of using the threshold and profit_functions

In [7]:
#List of probabilities
probs = np.array([0.2, 0.5, 0.8, 0.9, 0.1, 0.75])

#Pass in probabilities into threshold function, using .7 as threshold
preds = threshold(probs, thres=.7)
preds

array([0, 0, 1, 1, 0, 1])

In [8]:
sample_df = {"loan_amnt": [1000, 500, 200, 5000, 3000, 6000],
            "int_rate": [18, 20, 4, 5, 2, 10], 
            "target": [0, 1, 0, 1, 0, 1]}

#Put dictionary in data frame

profit_df = pd.DataFrame(sample_df)

profit_df

,int_rate,loan_amnt,target
0,18,1000,0
1,20,500,1
2,4,200,0
3,5,5000,1
4,2,3000,0
5,10,6000,1


In [8]:
#Add in predictions

profit_df["predicted"] = preds
profit_df

,int_rate,loan_amnt,target,predicted
0,18,1000,0,0
1,20,500,1,0
2,4,200,0,1
3,5,5000,1,1
4,2,3000,0,0
5,10,6000,1,1


In [11]:
#Apply function onto data frame
profit_series = profit_df.apply(profit_function, axis = 1)
profit_series

0      0.0
1      0.0
2   -200.0
3    250.0
4      0.0
5    600.0
dtype: float64

In [12]:
#Sum up profits and losses
profit_series.sum()

650.0

This model made $650

## Let the games begin!!

In [3]:
#Imports and set pandas options
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
pd.set_option("max.columns", 100)
pd.set_option("max.colwidth", 100)

In [18]:
# Load in training data.
# Loan_status column is the target variable. Remember to drop it from df.
train = pd.read_csv("../../data/lending_club/challenge_training_data.csv")
#train.head(2)
test = pd.read_csv("../../data/lending_club/challenge_testing_data.csv")
test.head(2)

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,acc_now_delinq,delinq_amnt,tax_liens
0,1077430,2500.0,2500.0,60 months,15.27%,59.83,C,Ryder,< 1 year,RENT,30000.0,Source Verified,0,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking a...,car,bike,1.0,0.0,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,Oct-2016,0.0,0.0,0.0
1,1076863,10000.0,10000.0,36 months,13.49%,339.31,C,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,1,"Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & ...",other,personel,20.0,0.0,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0,0.0,0.0


In [19]:
train.shape

(13689, 30)

In [20]:
test.shape

(13689, 30)

In [21]:
train.describe()

,id,loan_amnt,funded_amnt,installment,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,acc_now_delinq,delinq_amnt,tax_liens
count,1.368900e+04,13689.000000,13689.000000,13689.000000,1.368900e+04,13689.000000,13689.000000,13689.000000,13689.000000,13689.000000,13689.000000,13689.000000,13689.00000,13689.000000,13689.000000,13689.0,13689.0,13689.000000
mean,6.233684e+05,11364.750895,11085.603404,332.900659,6.979179e+04,0.792169,13.489158,0.151143,712.332530,716.332530,1.154869,9.478267,0.05793,14934.617357,22.224706,0.0,0.0,0.000073
std,2.224602e+05,7259.133486,7011.722526,207.102750,5.787344e+04,0.405770,6.743814,0.499875,35.851729,35.851729,1.657204,4.491987,0.24342,22599.094015,11.556404,0.0,0.0,0.008547
min,5.574200e+04,500.000000,500.000000,15.760000,3.600000e+03,0.000000,0.000000,0.000000,610.000000,614.000000,0.000000,1.000000,0.00000,0.000000,1.000000,0.0,0.0,0.000000
25%,4.537700e+05,6000.000000,5950.000000,173.860000,4.157900e+04,1.000000,8.340000,0.000000,685.000000,689.000000,0.000000,6.000000,0.00000,3885.000000,14.000000,0.0,0.0,0.000000
50%,5.958470e+05,10000.000000,10000.000000,291.930000,6.000000e+04,1.000000,13.600000,0.000000,710.000000,714.000000,1.000000,9.000000,0.00000,9198.000000,20.000000,0.0,0.0,0.000000
75%,7.921460e+05,15000.000000,15000.000000,444.790000,8.300000e+04,1.000000,18.830000,0.000000,735.000000,739.000000,2.000000,12.000000,0.00000,17819.000000,29.000000,0.0,0.0,0.000000
max,1.075358e+06,35000.000000,35000.000000,1288.100000,2.039784e+06,1.000000,29.990000,11.000000,825.000000,829.000000,28.000000,47.000000,3.00000,508961.000000,90.000000,0.0,0.0,1.000000


In [22]:
#null accuracy
train.loan_status.mean()

0.7921688947329973

In [23]:
train.dtypes

id                       int64
loan_amnt              float64
funded_amnt            float64
term                    object
int_rate                object
installment            float64
grade                   object
emp_title               object
emp_length              object
home_ownership          object
annual_inc             float64
verification_status     object
loan_status              int64
desc                    object
purpose                 object
title                   object
dti                    float64
delinq_2yrs            float64
fico_range_low         float64
fico_range_high        float64
inq_last_6mths         float64
open_acc               float64
pub_rec                float64
revol_bal              float64
revol_util              object
total_acc              float64
last_credit_pull_d      object
acc_now_delinq         float64
delinq_amnt            float64
tax_liens              float64
dtype: object

In [84]:
train.tax_liens.describe()

count    5.0
mean     0.0
std      0.0
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: tax_liens, dtype: float64

In [85]:
train.last_credit_pull_d.unique()

array(['Feb-2014', 'Feb-2017', 'Sep-2011'], dtype=object)

In [87]:
date_new = {'last_credit_pull_d' : {"Feb-2014": 1,'Feb-2017' : 2,'Sep-2011':3}}
train.replace(date_new, inplace=True)

In [24]:
# missing values
train.isnull().sum()

id                     0
loan_amnt              0
funded_amnt            0
term                   0
int_rate               0
installment            0
grade                  0
emp_title              0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
desc                   0
purpose                0
title                  0
dti                    0
delinq_2yrs            0
fico_range_low         0
fico_range_high        0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_acc              0
last_credit_pull_d     0
acc_now_delinq         0
delinq_amnt            0
tax_liens              0
dtype: int64

In [29]:
train = train.drop(labels = ['acc_now_delinq','delinq_amnt','id'], axis=1)

In [30]:
train.corr()

,loan_amnt,funded_amnt,installment,annual_inc,loan_status,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,tax_liens
loan_amnt,1.000000,0.979824,0.926593,0.302584,0.013136,0.073157,-0.034562,0.128810,0.128810,-0.053090,0.169631,-0.048453,0.254794,0.259847,-0.011027
funded_amnt,0.979824,1.000000,0.954544,0.300004,0.014521,0.074090,-0.037667,0.120513,0.120513,-0.053264,0.167923,-0.049016,0.253378,0.254285,-0.011076
installment,0.926593,0.954544,1.000000,0.310228,0.009810,0.065028,-0.025896,0.054176,0.054176,-0.029958,0.169073,-0.043203,0.275574,0.239929,-0.010941
annual_inc,0.302584,0.300004,0.310228,1.000000,0.032233,-0.121657,0.031826,0.045770,0.045770,0.026730,0.202448,-0.015188,0.339139,0.290302,-0.003662
loan_status,0.013136,0.014521,0.009810,0.032233,1.000000,-0.046280,-0.035297,0.238120,0.238120,-0.371061,-0.019613,-0.080022,-0.103399,0.007716,-0.016687
dti,0.073157,0.074090,0.065028,-0.121657,-0.046280,1.000000,-0.035435,-0.189935,-0.189935,0.004994,0.297160,-0.005538,0.206351,0.241738,-0.006007
delinq_2yrs,-0.034562,-0.037667,-0.025896,0.031826,-0.035297,-0.035435,1.000000,-0.220971,-0.220971,0.022363,0.018300,0.017498,-0.047559,0.067307,-0.002584
fico_range_low,0.128810,0.120513,0.054176,0.045770,0.238120,-0.189935,-0.220971,1.000000,1.000000,-0.149492,-0.028387,-0.161982,-0.035749,0.105756,-0.016053
fico_range_high,0.128810,0.120513,0.054176,0.045770,0.238120,-0.189935,-0.220971,1.000000,1.000000,-0.149492,-0.028387,-0.161982,-0.035749,0.105756,-0.016053
inq_last_6mths,-0.053090,-0.053264,-0.029958,0.026730,-0.371061,0.004994,0.022363,-0.149492,-0.149492,1.000000,0.093469,0.070665,0.002995,0.085250,0.035306


In [42]:
train.select_dtypes(include = ['object'])



,term,int_rate,grade,emp_title,emp_length,home_ownership,verification_status,desc,purpose,title,revol_util,last_credit_pull_d
0,36 months,10.99%,B,A-1 Casino Rd Mini Storage,9 years,RENT,Verified,Borrower added on 05/02/11 > I plan to use these funds to pay off a higher rate credit card.<b...,debt_consolidation,lowerinterest,76.4%,Feb-2014
1,36 months,7.51%,A,ORION MECHANICAL INC.,4 years,OWN,Not Verified,Borrower added on 08/19/10 > CONSOLIDATE OUT STANDING CREDIT CARD LOANS AND PURCHASES FOR NEW ...,debt_consolidation,CONSOLIDATE,2.9%,Feb-2017
2,36 months,12.53%,B,"U.S. Government, Dept of Homeland Securi",3 years,RENT,Not Verified,"568147 added on 11/04/09 > Personal Loan, Deposited to TD nov 04 2009<br/> 568147 added on 11/...",other,Personal Loan 11/2009,7.5%,Feb-2017
3,36 months,13.49%,C,Boeing,5 years,OWN,Verified,"I'll use this to finish paying off my credit cards. I do make $80k a year, so I expect that I'll...",debt_consolidation,LendingClubLoan,95.7%,Sep-2011
4,36 months,13.22%,C,Charles Wright Academy,4 years,RENT,Not Verified,"I am a single mother to a wonderful 7 year old girl. We have had our share of difficulties, most...",debt_consolidation,"Medical, Consolidation, & Kauai, Oh My!",88.7%,Feb-2017
5,36 months,11.99%,B,Mega Airlines Solutions,1 year,OWN,Source Verified,"Borrower added on 07/14/11 > I plan on using the funds if provided, to eliminate my debt and h...",debt_consolidation,Debt Free,92.5%,Nov-2015
6,36 months,11.34%,C,Best Buy,3 years,RENT,Not Verified,Looking to pay off some Credit Cards before we move into our new house and start working at our ...,debt_consolidation,Paying off Bills,100.7%,Feb-2017
7,36 months,13.99%,C,Winnetka Public School District 36,6 years,RENT,Not Verified,Borrower added on 07/06/11 > I am a tenured educator. This loan will be used to consolidate my...,debt_consolidation,Loan,90.7%,Nov-2013
8,36 months,12.12%,B,Southern Company,5 years,RENT,Not Verified,Borrower added on 06/10/11 > This loan will be used to pay off a credit card with a higher rat...,credit_card,Credit Card Refinance,67%,Feb-2014
9,60 months,16.32%,D,nw mutual,3 years,RENT,Not Verified,Borrower added on 07/13/10 > Just trying to take my fiancee on a trip to the bahammas i dont r...,major_purchase,vacation,60.6%,Oct-2016


In [43]:
train.emp_length.unique()

array(['9 years', '4 years', '3 years', '5 years', '1 year', '6 years',
       '< 1 year', '10+ years', '7 years', '8 years', '2 years', 'n/a'], dtype=object)

In [88]:
train.emp_length.mode()

2.2000000000000002

In [79]:
train.loc[train.emp_length == 'n/a'].fillna((train.emp_length.mean()),inplace = True)

,loan_amnt,funded_amnt,term,int_rate,installment,emp_title,emp_length,annual_inc,loan_status,desc,title,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,tax_liens,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G


In [81]:
train.emp_length.value_counts()

2    4
3    1
Name: emp_length, dtype: int64

In [44]:
train.verification_status.unique()

array(['Verified', 'Not Verified', 'Source Verified'], dtype=object)

In [50]:
train.home_ownership.unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE'], dtype=object)

In [56]:
emp_length_new = {'emp_length' : {"< 1 year": 0,'1 year' : 2,'2 years':2,'3 years':2,'4 years':2,'5 years': 2,
                                   '6 years':3,'7 years':3,'9 years':3,'8 years':3,'10 years': 3,"10+ years" : 4}}

In [51]:
train.purpose.unique()

array(['debt_consolidation', 'other', 'credit_card', 'major_purchase',
       'small_business', 'home_improvement', 'car', 'house', 'wedding',
       'moving', 'vacation', 'medical', 'renewable_energy', 'educational'], dtype=object)

In [57]:
train.replace(emp_length_new, inplace=True)
train.head()

,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,tax_liens
0,3150.0,3150.0,36 months,10.99%,103.12,B,A-1 Casino Rd Mini Storage,3,RENT,15300.0,Verified,1,Borrower added on 05/02/11 > I plan to use these funds to pay off a higher rate credit card.<b...,debt_consolidation,lowerinterest,19.76,0.0,695.0,699.0,0.0,5.0,0.0,9316.0,76.4%,9.0,Feb-2014,0.0
1,10000.0,10000.0,36 months,7.51%,311.11,A,ORION MECHANICAL INC.,2,OWN,125000.0,Not Verified,1,Borrower added on 08/19/10 > CONSOLIDATE OUT STANDING CREDIT CARD LOANS AND PURCHASES FOR NEW ...,debt_consolidation,CONSOLIDATE,18.43,0.0,800.0,804.0,1.0,16.0,0.0,25364.0,2.9%,40.0,Feb-2017,0.0
2,11000.0,11000.0,36 months,12.53%,368.13,B,"U.S. Government, Dept of Homeland Securi",2,RENT,81600.0,Not Verified,1,"568147 added on 11/04/09 > Personal Loan, Deposited to TD nov 04 2009<br/> 568147 added on 11/...",other,Personal Loan 11/2009,5.12,0.0,725.0,729.0,2.0,5.0,0.0,358.0,7.5%,10.0,Feb-2017,0.0
3,2200.0,2200.0,36 months,13.49%,74.65,C,Boeing,2,OWN,80000.0,Verified,1,"I'll use this to finish paying off my credit cards. I do make $80k a year, so I expect that I'll...",debt_consolidation,LendingClubLoan,3.27,0.0,700.0,704.0,0.0,2.0,0.0,10142.0,95.7%,6.0,Sep-2011,0.0
4,4500.0,4500.0,36 months,13.22%,152.11,C,Charles Wright Academy,2,RENT,30555.0,Not Verified,1,"I am a single mother to a wonderful 7 year old girl. We have had our share of difficulties, most...",debt_consolidation,"Medical, Consolidation, & Kauai, Oh My!",4.32,0.0,690.0,694.0,1.0,4.0,1.0,3371.0,88.7%,14.0,Feb-2017,0.0


In [74]:
train = pd.get_dummies(train, columns=["home_ownership","verification_status","purpose","grade"]).head()

In [75]:
train['term'] = train['term'].str.rstrip('months').astype('float').head()

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [64]:
train['revol_util'] = train['revol_util'].str.rstrip('%').astype('float')

In [68]:
train['int_rate'] = train['int_rate'].str.rstrip('%').astype('float').head()

In [76]:
train.dtypes

loan_amnt                              float64
funded_amnt                            float64
term                                   float64
int_rate                               float64
installment                            float64
emp_title                               object
emp_length                              object
annual_inc                             float64
loan_status                              int64
desc                                    object
title                                   object
dti                                    float64
delinq_2yrs                            float64
fico_range_low                         float64
fico_range_high                        float64
inq_last_6mths                         float64
open_acc                               float64
pub_rec                                float64
revol_bal                              float64
revol_util                             float64
total_acc                              float64
last_credit_p

In [89]:
train.last_credit_pull_d.unique()

array([1, 2, 3])